In [11]:
import itertools

In [15]:
# Below I implement a suffix tree to calculate the longest common substring
# of a list with two or more strings
# Time complexity:  O(sum of lengths of strings) i.e. O(len(string_1)+len(string_2)+...len(string_n))


class suffixNode():
  def __init__(self, Index=-1, ParentNode=None, Depth=-1):
    self.s_link = None
    self.transitions = {}
    self.Index = Index
    self.Depth = Depth
    self.Parent = ParentNode
    self.gen_id = {}

  def transition(self, suffix):
    return suffix in self.transitions

  def goThru(self, f):
    for node in self.transitions.values():
      node.goThru(f)
    f(self)

  def t_link(self, suffix):
    if suffix not in self.transitions:
      return False
    else:
      return self.transitions[suffix]

  def get_S(self):
    if self.s_link is not None:
      return self.s_link
    else:
      return False

class suffixTree():
  def __init__(self, inp=''):
      self.rt = suffixNode()
      self.rt.Index = 0
      self.rt.Depth = 0
      self.rt.s_link = self.rt
      self.rt.Parent = self.rt
      if not inp == '':
        self.create(inp)

  def validate(self, inp):
      if isinstance(inp, list):
        if all(isinstance(item, str) for item in inp):
          return('proceed')
      raise ValueError("input should be a list of strings")

  def terminalGen(self):
    spl = list(list(range(0x100000, 0x10FFFD+1)) + list(range(0xF0000, 0xFFFFD+1)) + list(range(0xE000, 0xF8FF+1)))
    for i in spl:
      yield (chr(i))
    raise ValueError("too many strings in the input")

  def create(self, alpha):
      type = self.validate(alpha)
      if type == 'proceed':
        self.createHelper2(alpha)

  def createHelper(self, alpha):
      self.word = alpha
      self.algoCreate(alpha) 

  def createHelper2(self, addTerminal):
    terminal_gen = self.terminalGen()
    _addTerminal = ''.join([alpha + next(terminal_gen) for alpha in addTerminal])
    self.word = _addTerminal
    self.beginWord = []
    i = 0
    for n in range(len(addTerminal)):
      self.beginWord.append(i)
      i += len(addTerminal[n]) + 1
    self.createHelper(_addTerminal)
    self.rt.goThru(self.labels)
  
  # create the suffix tree using McCreight's algorithm 
  def algoCreate(self, alpha):
      a = self.rt
      d = 0
      for i in range(len(alpha)):
        while a.Depth == d and a.transition(alpha[d + i]):
          a = a.t_link(alpha[d + i])
          d += 1
          while d < a.Depth and alpha[a.Index + d] == alpha[i + d]:
            d += 1
        if d < a.Depth:
          a = self.createHelperN(alpha, a, d)
        self.createHelperL(alpha, i, a, d)
        if not a.get_S():
          self.slink(alpha, a)
        a = a.get_S()
        d -= 1
        if d < 0:
          d = 0

  def createHelperL(self, alpha, i, a, d):
    n1 = suffixNode()
    n1.Index = i
    n1.Depth = len(alpha) - i
    a.transitions[alpha[i+d]] = n1
    n1.Parent = a
    return n1
  
  def createHelperN(self, alpha, a, d):
    i = a.Index
    p = a.Parent
    b = suffixNode(Index=i, Depth=d)
    b.transitions[alpha[i+d]] = a
    a.Parent = b
    p.transitions[alpha[i+p.Depth]] = b
    b.Parent = p
    return b

  def slink(self, alpha, a):
    d = a.Depth
    b = a.Parent.get_S()
    while b.Depth < d - 1:
      b = b.t_link(alpha[a.Index + b.Depth + 1])
    if b.Depth > d - 1:
      b = self.createHelperN(alpha, b, d - 1)
    a.s_link = b

  def labels(self, node):
    if len(node.transitions) == 0:
      alpha = {self.start(node.Index)}
    else:
      alpha = {n for ns in node.transitions.values() for n in ns.gen_id}
    node.gen_id = alpha

  def start(self, Index):
    i = 0
    for ind in self.beginWord[1:]:
      if Index < ind:
        return i
      else:
        i += 1
    return i

  def lcs(self, str_id=-1):
    if str_id == -1 or not isinstance(str_id, list):
      str_id = set(range(len(self.beginWord)))
    else:
      str_id = set(str_id)

    deep_nd = self.lcsHelper(self.rt, str_id)
    start = deep_nd.Index
    end = deep_nd.Index + deep_nd.Depth
    return self.word[start:end]

  def lcsHelper(self, node, str_id):
    nd = [self.lcsHelper(n, str_id) for n in node.transitions.values() if n.gen_id.issuperset(str_id)]
    if nd == []:
        return node
    deep_nd = max(nd, key=lambda n: n.Depth)
    return deep_nd


In [16]:
# read the sample files (be sure to upload them before running this code)
# change this code if you are running it on a different set of files so as to point to the correct file names
a = {}
for i in range(1, 11):
  with open('sample.'+str(i), mode='rb') as file:
    fileContent = file.read()
    a['sample.'+str(i)] = str(fileContent)


In [17]:
def combinationsOfTwo(iterable):
  # create a list of n choose 2 combinations of the sample files
  return list(itertools.combinations(iterable, 2))

combinationsOfTwo = list(combinationsOfTwo(a))

combinationsOfTwoWithLength = {}
for i in range(0, len(combinationsOfTwo)):
  # sum up the lengths of the two files in the combination
  combinationsOfTwoWithLength[len(a[combinationsOfTwo[i][0]])+ len(a[combinationsOfTwo[i][1]])] = [combinationsOfTwo[i]]

# dictionary with combinations sorted by the sum of length of the two individual files
combinationsOfTwoWithLength_sorted = dict(sorted(combinationsOfTwoWithLength.items())) 

# descending order of sum of length of both files
largest_length_2_combinations = []
tmp = list(combinationsOfTwoWithLength_sorted.values())[::-1]
for i in range(0, len(combinationsOfTwoWithLength_sorted)):
  largest_length_2_combinations.append(tmp[i][0])

longest = 0
longest_lcs = ''

# We iterate down through the combinations of the files with the longest combined length
# and keep checking when the size of the longest LCS becomes greater than the
# smaller of the two files.
# For me, the below code takes around 12 seconds to run on the 10 sample files using the GPU accelerator on Google Colab
for i in range(0, len(largest_length_2_combinations)):
  # print(i)
  subset = largest_length_2_combinations[i]
  subset_strings = []
  for j in range(0, len(subset)):
    subset_strings.append(a[subset[j]])
  len_smallest_string = len(min(subset_strings, key = len)) # length of the smaller of the two files
  if len(longest_lcs) >= len_smallest_string: # if the LCS is already larger than the smaller of the two files, stop checking for new LCS's 
  # (assumes no tie breaking)
    break
  st = suffixTree(list(subset_strings)) # create the suffix tree
  lcs = st.lcs() # call the LCS function to find the longest common substring among the two files
  if len(lcs) > longest:
    longest = len(lcs)
    longest_lcs = lcs

print('length of longest strand in 2 or more files:', len(longest_lcs))
print('files in which the longest strand was present, and the offset where the longest strand appears in them:')
for i in range(1, len(a)):
  if list(a.values())[i].find(longest_lcs) >= 0:
    print(list(a.keys())[i], ',', list(a.values())[i].find(longest_lcs))

length of longest strand in 2 or more files: 79670
files in which the longest strand was present, and the offset where the longest strand appears in them:
sample.2 , 8971
sample.3 , 50214
